In [2]:
import sys, os
sys.path.append('/Users/benjamin/Desktop/repos/chi-data/backend/aws') 
sys.path.append('/Users/benjamin/Desktop/repos/chi-data/backend') 
from s3 import S3
from dynamo import DynamoConn
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
from config import cook_tracts, chicago_tracts, msa_tracts
# https://www.dnainfo.com/chicago/20171020/downtown/chicago-amazon-headquarters

In [3]:
import s3fs
s3 = s3fs.S3FileSystem()
dataset = 'JT00'
year = '2014'
ds = pq.ParquetDataset(path_or_paths=[
  'bnroths/chicago-data/lehd_od/year=%s/il_lehd_od_main_%s_%s.parquet' % (year, dataset, year), 
  'bnroths/chicago-data/lehd_od/year=%s/il_lehd_od_aux_%s_%s.parquet' % (year, dataset, year), 
    
  'bnroths/chicago-data/lehd_od/year=%s/in_od_main_%s_%s.parquet' % (year, dataset, year), 
  'bnroths/chicago-data/lehd_od/year=%s/in_od_aux_%s_%s.parquet' % (year, dataset, year), 
    
  'bnroths/chicago-data/lehd_od/year=%s/wi_od_main_%s_%s.parquet' % (year, dataset, year), 
  'bnroths/chicago-data/lehd_od/year=%s/wi_od_aux_%s_%s.parquet' % (year, dataset, year), 

], filesystem=s3, validate_schema=False)

In [4]:
table = ds.read()
df = table.to_pandas()


In [5]:
df.head()


,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,h_tract,w_tract
0,170010001001001,170010001001065,1,0,1,0,0,1,0,0,0,1,20160219,17001000100,17001000100
1,170010001001001,170010001002002,1,0,0,1,0,0,1,0,0,1,20160219,17001000100,17001000100
2,170010001001001,170010001003012,1,0,1,0,0,1,0,0,0,1,20160219,17001000100,17001000100
3,170010001001001,170010006003003,1,0,0,1,0,1,0,0,0,1,20160219,17001000600,17001000100
4,170010001001001,170010008001007,1,0,0,1,0,1,0,0,0,1,20160219,17001000800,17001000100


In [6]:
df.head()
df = df[df['h_tract'].isin(msa_tracts)]
df = df[df['w_tract'].isin(msa_tracts)]

In [7]:
diff1 = set(df.h_tract) - set(df.w_tract)
for tract in diff1:
	df = df[df.h_tract != tract]

diff2 = set(df.w_tract) - set(df.h_tract)
for tract in diff2:
	df = df[df.w_tract != tract]

In [8]:
diff1 = set(df.h_tract) - set(df.w_tract)
diff2 = set(df.w_tract) - set(df.h_tract)
print diff1, diff2
print df.head()
df.shape

set([]) set([])
              w_geocode        h_geocode  S000  SA01  SA02  SA03  SE01  SE02  \
189043  170310101001000  170310101002006     1     0     1     0     0     1   
189044  170310101001000  170310101003006     1     0     0     1     0     0   
189045  170310101001000  170310103003000     1     0     0     1     0     0   
189046  170310101001000  170310105022000     1     0     1     0     0     0   
189047  170310101001000  170310317001000     1     0     1     0     0     1   

        SE03  SI01  SI02  SI03  createdate      h_tract      w_tract  
189043     0     0     0     1    20160219  17031010100  17031010100  
189044     1     0     0     1    20160219  17031010100  17031010100  
189045     1     0     0     1    20160219  17031010300  17031010100  
189046     1     0     0     1    20160219  17031010502  17031010100  
189047     0     0     0     1    20160219  17031031700  17031010100  


(3727688, 15)

In [22]:
pivot = pd.pivot_table(df, values='SA01', columns=['w_tract'], index=['h_tract'], aggfunc=np.sum, fill_value=0)

In [23]:
print pivot.head()
pivot.shape

# df['S000'].sum()#['17001010400']#.head(n=100) #17001010100

w_tract      17031010100  17031010201  17031010202  17031010300  17031010400  \
h_tract                                                                        
17031010100            8            6            9            7            0   
17031010201            2            4            5            3            0   
17031010202            0            0            4            0            1   
17031010300            3            0            7            5            3   
17031010400            0            0            0            2            0   

w_tract      17031010501  17031010502  17031010503  17031010600  17031010701  \
h_tract                                                                        
17031010100            0            0            0            2            0   
17031010201            2            0            0            1            0   
17031010202            0            0            0            0            0   
17031010300            3            0  

(2204, 2204)

In [81]:

pivot.to_csv('/Users/benjamin/Desktop/repos/chi-data/scripts/test.csv')

In [111]:
h_tracts = pivot.sum()
print h_tracts.head()
w_tracts = pivot.transpose().sum()
A = pivot.transpose()/w_tracts
A = A.fillna(0).replace([np.inf, -np.inf], 0)

w_tract
17031010100     85
17031010201     57
17031010202    186
17031010300    200
17031010400    556
dtype: int64


In [113]:
orig_h_tracts = np.matmul(A, h_tracts)

In [114]:
orig_h_tracts

array([  44.05010717,   40.51681746,  104.08125039, ...,   38.18237595,
          3.73307171,   70.14956773])

In [115]:
for n in ['17031841900', '17031833300', '17031330200', '17031839000']:
    print n, h_tracts[n]
    h_tracts[n] = h_tracts[n] * 10
    

17031841900 5587
17031833300 305
17031330200 87
17031839000 585


In [106]:
#w_tracts.sum()
w_tracts.sum()
#
880178 - 864518

15660

In [107]:
new_w_tracts = np.matmul(A, h_tracts)

In [110]:
new_h_tracts

array([  94.,   57.,  186., ...,   70.,   10.,  181.])

In [116]:
diff = new_h_tracts - orig_h_tracts
from s3 import S3
s3 = S3()
tracts = {}
for i, column in enumerate(h_tracts):
    tracts[h_tracts.index[i]] = {"real": 1000* diff[i]}

final = {
    "data": tracts,
    "meta": {
        # "max": abs(max(tracts.values(), key=abs)),
        "max": max(tracts.values()),
        "min": min(tracts.values()),
        "top": sorted(tracts.values())[-15]
    }
}
import json
with open('amazon.json', 'w') as f:
    f.write(json.dumps(final))
    
s3.save_file_public(
    local='amazon.json',
    dataset='amazon-opposite', 
    dt='2002', 
    filename='all.json'
)

True

In [109]:
#len(diff)
#diff.sum()
408 + 224 + 279 + 829
diff.sum()
#new_h_tracts
#new_h_tracts.sum() - orig_h_tracts.sum()

15660.0

In [75]:
for i, col in enumerate(h_tracts[:1]):
    print i, col, h_tracts.index[i]
h_tracts.index

0 85 17031010100


Index([u'17031010100', u'17031010201', u'17031010202', u'17031010300',
       u'17031010400', u'17031010501', u'17031010502', u'17031010503',
       u'17031010600', u'17031010701',
       ...
       u'55059002601', u'55059002602', u'55059002700', u'55059002800',
       u'55059002903', u'55059002904', u'55059002905', u'55059002906',
       u'55059003001', u'55059003002'],
      dtype='object', name=u'w_tract', length=2204)